<a href="https://colab.research.google.com/github/RishabhG998/Applied-Machine-Learning/blob/main/Generative_AI_Use_Case_Summarize_Dialogue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 52.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 re

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## Summarize Dialogue without Prompt Engineering


##### in this use case, we will be generating a summary of a dialogue with the pre-trained Large Language Model (LLM) FLAN-T5 from hugging Face. The list of available models in the Hugging Face transformers package can be found on their website.

##### Upload dialogue from DialogSum Hugging Face dataset. It contains 10000+ dialogues with the corresponding manually labeled summaries and topics.

In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
examples_indices = [40, 200]

dash = '-'.join('' for x in range(100))

for i, index in enumerate(examples_indices):
  print(dash)
  print('Example ', i+1)
  print(dash)
  print('INPUT DIALOGUE')
  print(dataset['test'][index]['dialogue'])
  print(dash)
  print('BASELINE HUMAN SUMMARY: ')
  print(dash)
  print(dataset['test'][index]['summary'])


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
---------------------------------------------------------------------------------------------------
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Exam

##### Load the FLAN-T5 model, from an instance of AutoModelForSeq2SeqLM class with the .from_pretrained() method

In [5]:
model_name = 'google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

(…)le/flan-t5-base/resolve/main/config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

(…)base/resolve/main/generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

##### Tokenization is the process of splitting texts into smaller units that can be processed by the LLM models.

##### Download the tokenizer for FLAN-T5 model using AutoTokenizer.from_pretrained() method. use_fast parameter switches on fast tokenizer.

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

(…)-base/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…)flan-t5-base/resolve/main/tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

(…)ase/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

##### Test the tokenizer with a simple sentence

In [7]:
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
    sentence_encoded["input_ids"][0],
    skip_special_tokens = True
)

print('ENCODED SENTENCE: ')
print(sentence_encoded['input_ids'][0])
print('\nDECODED SENTENCE: ')
print(sentence_decoded)

ENCODED SENTENCE: 
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

DECODED SENTENCE: 
What time is it, Tom?


##### Trying the sentence on the LLM for summarization without prompt engineering. Prompt engineering is the act of human changing the prompt (input) to improve the response of the LLM on a given task.

In [8]:
for i, index in enumerate(examples_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  inputs = tokenizer(dialogue, return_tensors='pt')
  print("input:\n ", inputs, '\n\n\n')
  # print("input_ids:\n ", inputs["input_ids"])
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )

  print(dash)
  print('Example ', i+1)
  print(dash)
  print(f'INPUT PROMPT:\n{dialogue}')
  print(dataset['test'][index]['dialogue'])
  print(dash)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash)
  print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')


input:
  {'input_ids': tensor([[ 1713,   345, 13515,   536,  4663,    10,   363,    97,    19,    34,
             6,  3059,    58,  1713,   345, 13515,   357,  4663,    10,  1142,
             3,     9,  1962,     5,    94,    31,     7,     3,   324,    12,
          4169,    57,    82,  1605,     5,  1713,   345, 13515,   536,  4663,
            10,    27,     7,    34,    58,    27,   141,   150,   800,    34,
            47,    78,  1480,     5,    27,   398,    36,   326,   230,     5,
          1713,   345, 13515,   357,  4663,    10,   363,    31,     7,     8,
         23601,    58,  1713,   345, 13515,   536,  4663,    10,    27,   398,
          3579,     8,  4169,    18,    17,  9288,    17,    63,  2412,     5,
          1713,   345, 13515,   357,  4663,    10,   148,    31,   162,  2500,
            13,    97,   780,     5,    37, 14421,  2478,    19,   182,   885,
             5,    94,   751,    31,    17,   240,    72,   145,  6786,   676,
            12,   129,   132,

#### Summarize Dialogue with an Instruction Prompt
Prompt engineering is an important concept in using foundation models for text generation.

##### Zero Shot Inference with an Instruction Prompt
In order to instruct the model to perform a task - summarize a dialogue - we can take the dialogue and convert it into an instruction prompt. This is often called
zero shot inference.

Wrap the dialogue in a descriptive instruction and see how the generated text will change:

In [16]:
for i, index in enumerate(examples_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  summarize the the given conversation:

  {dialogue}

  Summary:
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  print("input:\n ", inputs, '\n\n\n')
  # print("input_ids:\n ", inputs["input_ids"])
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )

  print(dash)
  print('Example ', i+1)
  print(dash)
  print(f'INPUT PROMPT:\n{prompt}')
  # print(dataset['test'][index]['dialogue'])
  print(dash)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash)
  print(f'MODEL GENERATION - WITH ZERO SHOT INFERENCE:\n{output}\n')


input:
  {'input_ids': tensor([[21603,     8,     8,   787,  3634,    10,  1713,   345, 13515,   536,
          4663,    10,   363,    97,    19,    34,     6,  3059,    58,  1713,
           345, 13515,   357,  4663,    10,  1142,     3,     9,  1962,     5,
            94,    31,     7,     3,   324,    12,  4169,    57,    82,  1605,
             5,  1713,   345, 13515,   536,  4663,    10,    27,     7,    34,
            58,    27,   141,   150,   800,    34,    47,    78,  1480,     5,
            27,   398,    36,   326,   230,     5,  1713,   345, 13515,   357,
          4663,    10,   363,    31,     7,     8, 23601,    58,  1713,   345,
         13515,   536,  4663,    10,    27,   398,  3579,     8,  4169,    18,
            17,  9288,    17,    63,  2412,     5,  1713,   345, 13515,   357,
          4663,    10,   148,    31,   162,  2500,    13,    97,   780,     5,
            37, 14421,  2478,    19,   182,   885,     5,    94,   751,    31,
            17,   240,    72,

In [17]:
for i, index in enumerate(examples_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  understand the dialogue below and create a summary of the conversation

  {dialogue}

  Summary:
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  print("input:\n ", inputs, '\n\n\n')
  # print("input_ids:\n ", inputs["input_ids"])
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )

  print(dash)
  print('Example ', i+1)
  print(dash)
  print(f'INPUT PROMPT:\n{prompt}')
  # print(dataset['test'][index]['dialogue'])
  print(dash)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash)
  print(f'MODEL GENERATION - WITH ZERO SHOT INFERENCE:\n{output}\n')


input:
  {'input_ids': tensor([[  734,     8,  7478,   666,    11,   482,     3,     9,  9251,    13,
             8,  3634,  1713,   345, 13515,   536,  4663,    10,   363,    97,
            19,    34,     6,  3059,    58,  1713,   345, 13515,   357,  4663,
            10,  1142,     3,     9,  1962,     5,    94,    31,     7,     3,
           324,    12,  4169,    57,    82,  1605,     5,  1713,   345, 13515,
           536,  4663,    10,    27,     7,    34,    58,    27,   141,   150,
           800,    34,    47,    78,  1480,     5,    27,   398,    36,   326,
           230,     5,  1713,   345, 13515,   357,  4663,    10,   363,    31,
             7,     8, 23601,    58,  1713,   345, 13515,   536,  4663,    10,
            27,   398,  3579,     8,  4169,    18,    17,  9288,    17,    63,
          2412,     5,  1713,   345, 13515,   357,  4663,    10,   148,    31,
           162,  2500,    13,    97,   780,     5,    37, 14421,  2478,    19,
           182,   885,     5,

In [18]:
for i, index in enumerate(examples_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  Dialogue:

  {dialogue}

  what was going on?
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  print("input:\n ", inputs, '\n\n\n')
  # print("input_ids:\n ", inputs["input_ids"])
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )

  print(dash)
  print('Example ', i+1)
  print(dash)
  print(f'INPUT PROMPT:\n{prompt}')
  # print(dataset['test'][index]['dialogue'])
  print(dash)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash)
  print(f'MODEL GENERATION - WITH ZERO SHOT INFERENCE:\n{output}\n')


input:
  {'input_ids': tensor([[ 5267, 10384,    10,  1713,   345, 13515,   536,  4663,    10,   363,
            97,    19,    34,     6,  3059,    58,  1713,   345, 13515,   357,
          4663,    10,  1142,     3,     9,  1962,     5,    94,    31,     7,
             3,   324,    12,  4169,    57,    82,  1605,     5,  1713,   345,
         13515,   536,  4663,    10,    27,     7,    34,    58,    27,   141,
           150,   800,    34,    47,    78,  1480,     5,    27,   398,    36,
           326,   230,     5,  1713,   345, 13515,   357,  4663,    10,   363,
            31,     7,     8, 23601,    58,  1713,   345, 13515,   536,  4663,
            10,    27,   398,  3579,     8,  4169,    18,    17,  9288,    17,
            63,  2412,     5,  1713,   345, 13515,   357,  4663,    10,   148,
            31,   162,  2500,    13,    97,   780,     5,    37, 14421,  2478,
            19,   182,   885,     5,    94,   751,    31,    17,   240,    72,
           145,  6786,   676,

In [19]:
for i, index in enumerate(examples_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  {dialogue}

  Act as if you are listening to this conversation and create a summary

  Summary:
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  print("input:\n ", inputs, '\n\n\n')
  # print("input_ids:\n ", inputs["input_ids"])
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )

  print(dash)
  print('Example ', i+1)
  print(dash)
  print(f'INPUT PROMPT:\n{prompt}')
  # print(dataset['test'][index]['dialogue'])
  print(dash)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash)
  print(f'MODEL GENERATION - WITH ZERO SHOT INFERENCE:\n{output}\n')


input:
  {'input_ids': tensor([[ 1713,   345, 13515,   536,  4663,    10,   363,    97,    19,    34,
             6,  3059,    58,  1713,   345, 13515,   357,  4663,    10,  1142,
             3,     9,  1962,     5,    94,    31,     7,     3,   324,    12,
          4169,    57,    82,  1605,     5,  1713,   345, 13515,   536,  4663,
            10,    27,     7,    34,    58,    27,   141,   150,   800,    34,
            47,    78,  1480,     5,    27,   398,    36,   326,   230,     5,
          1713,   345, 13515,   357,  4663,    10,   363,    31,     7,     8,
         23601,    58,  1713,   345, 13515,   536,  4663,    10,    27,   398,
          3579,     8,  4169,    18,    17,  9288,    17,    63,  2412,     5,
          1713,   345, 13515,   357,  4663,    10,   148,    31,   162,  2500,
            13,    97,   780,     5,    37, 14421,  2478,    19,   182,   885,
             5,    94,   751,    31,    17,   240,    72,   145,  6786,   676,
            12,   129,   132,

In [20]:
for i, index in enumerate(examples_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  {dialogue}

  What do you understand from the dialogue?
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  print("input:\n ", inputs, '\n\n\n')
  # print("input_ids:\n ", inputs["input_ids"])
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )

  print(dash)
  print('Example ', i+1)
  print(dash)
  print(f'INPUT PROMPT:\n{prompt}')
  # print(dataset['test'][index]['dialogue'])
  print(dash)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash)
  print(f'MODEL GENERATION - WITH ZERO SHOT INFERENCE:\n{output}\n')


input:
  {'input_ids': tensor([[ 1713,   345, 13515,   536,  4663,    10,   363,    97,    19,    34,
             6,  3059,    58,  1713,   345, 13515,   357,  4663,    10,  1142,
             3,     9,  1962,     5,    94,    31,     7,     3,   324,    12,
          4169,    57,    82,  1605,     5,  1713,   345, 13515,   536,  4663,
            10,    27,     7,    34,    58,    27,   141,   150,   800,    34,
            47,    78,  1480,     5,    27,   398,    36,   326,   230,     5,
          1713,   345, 13515,   357,  4663,    10,   363,    31,     7,     8,
         23601,    58,  1713,   345, 13515,   536,  4663,    10,    27,   398,
          3579,     8,  4169,    18,    17,  9288,    17,    63,  2412,     5,
          1713,   345, 13515,   357,  4663,    10,   148,    31,   162,  2500,
            13,    97,   780,     5,    37, 14421,  2478,    19,   182,   885,
             5,    94,   751,    31,    17,   240,    72,   145,  6786,   676,
            12,   129,   132,

In [22]:
for i, index in enumerate(examples_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  {dialogue}

  What are they discussing about? describe it.
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  print("input:\n ", inputs, '\n\n\n')
  # print("input_ids:\n ", inputs["input_ids"])
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )

  print(dash)
  print('Example ', i+1)
  print(dash)
  print(f'INPUT PROMPT:\n{prompt}')
  # print(dataset['test'][index]['dialogue'])
  print(dash)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash)
  print(f'MODEL GENERATION - WITH ZERO SHOT INFERENCE:\n{output}\n')


input:
  {'input_ids': tensor([[ 1713,   345, 13515,   536,  4663,    10,   363,    97,    19,    34,
             6,  3059,    58,  1713,   345, 13515,   357,  4663,    10,  1142,
             3,     9,  1962,     5,    94,    31,     7,     3,   324,    12,
          4169,    57,    82,  1605,     5,  1713,   345, 13515,   536,  4663,
            10,    27,     7,    34,    58,    27,   141,   150,   800,    34,
            47,    78,  1480,     5,    27,   398,    36,   326,   230,     5,
          1713,   345, 13515,   357,  4663,    10,   363,    31,     7,     8,
         23601,    58,  1713,   345, 13515,   536,  4663,    10,    27,   398,
          3579,     8,  4169,    18,    17,  9288,    17,    63,  2412,     5,
          1713,   345, 13515,   357,  4663,    10,   148,    31,   162,  2500,
            13,    97,   780,     5,    37, 14421,  2478,    19,   182,   885,
             5,    94,   751,    31,    17,   240,    72,   145,  6786,   676,
            12,   129,   132,

##### Building a function that takes example_indices_full, generates a prompt with examples, appends it to prompt which model needs to complete (example_to_summarize)

In [34]:
def prompt_engineering(example_indices_full, example_to_summarize):
  prompt=''
  for index in example_indices_full:
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    # the stop sequence '{summary}\n\n\n' is essential for FLAN-T5
    prompt +=f"""
    Dialogue:

    {dialogue}

    what was going on?

    {summary}


    """

  dialogue = dataset['test'][example_to_summarize]['dialogue']

  prompt +=f"""
  Dialogue:

  {dialogue}

  what was going on?
  """
  return prompt

In [37]:
example_indices_full = [40]
example_to_summarize = 200

one_shot_prompt = prompt_engineering(example_indices_full,example_to_summarize)
print(one_shot_prompt)


    Dialogue:

    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    what was going on?

    #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


    
  Dialogue:

  #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to be

In [38]:

summary = dataset['test'][example_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
print("input:\n ", inputs, '\n\n\n')
# print("input_ids:\n ", inputs["input_ids"])
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash)
print('Example ', i+1)
print(dash)
print(f'INPUT PROMPT:\n{one_shot_prompt}')
# print(dataset['test'][index]['dialogue'])
print(dash)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash)
print(f'MODEL GENERATION - WITH ONE SHOT INFERENCE:\n{output}\n')


input:
  {'input_ids': tensor([[ 5267, 10384,    10,  1713,   345, 13515,   536,  4663,    10,   363,
            97,    19,    34,     6,  3059,    58,  1713,   345, 13515,   357,
          4663,    10,  1142,     3,     9,  1962,     5,    94,    31,     7,
             3,   324,    12,  4169,    57,    82,  1605,     5,  1713,   345,
         13515,   536,  4663,    10,    27,     7,    34,    58,    27,   141,
           150,   800,    34,    47,    78,  1480,     5,    27,   398,    36,
           326,   230,     5,  1713,   345, 13515,   357,  4663,    10,   363,
            31,     7,     8, 23601,    58,  1713,   345, 13515,   536,  4663,
            10,    27,   398,  3579,     8,  4169,    18,    17,  9288,    17,
            63,  2412,     5,  1713,   345, 13515,   357,  4663,    10,   148,
            31,   162,  2500,    13,    97,   780,     5,    37, 14421,  2478,
            19,   182,   885,     5,    94,   751,    31,    17,   240,    72,
           145,  6786,   676,

##### Trying out few shots inference

In [39]:
example_indices_full = [40,80,120]
example_to_summarize = 200

few_shot_prompt = prompt_engineering(example_indices_full,example_to_summarize)
print(few_shot_prompt)


    Dialogue:

    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    what was going on?

    #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


    
    Dialogue:

    #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic 

In [40]:
summary = dataset['test'][example_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
print("input:\n ", inputs, '\n\n\n')
# print("input_ids:\n ", inputs["input_ids"])
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash)
print('Example ', i+1)
print(dash)
print(f'INPUT PROMPT:\n{few_shot_prompt}')
# print(dataset['test'][index]['dialogue'])
print(dash)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash)
print(f'MODEL GENERATION - WITH ONE SHOT INFERENCE:\n{output}\n')


Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


input:
  {'input_ids': tensor([[ 5267, 10384,    10,  1713,   345, 13515,   536,  4663,    10,   363,
            97,    19,    34,     6,  3059,    58,  1713,   345, 13515,   357,
          4663,    10,  1142,     3,     9,  1962,     5,    94,    31,     7,
             3,   324,    12,  4169,    57,    82,  1605,     5,  1713,   345,
         13515,   536,  4663,    10,    27,     7,    34,    58,    27,   141,
           150,   800,    34,    47,    78,  1480,     5,    27,   398,    36,
           326,   230,     5,  1713,   345, 13515,   357,  4663,    10,   363,
            31,     7,     8, 23601,    58,  1713,   345, 13515,   536,  4663,
            10,    27,   398,  3579,     8,  4169,    18,    17,  9288,    17,
            63,  2412,     5,  1713,   345, 13515,   357,  4663,    10,   148,
            31,   162,  2500,    13,    97,   780,     5,    37, 14421,  2478,
            19,   182,   885,     5,    94,   751,    31,    17,   240,    72,
           145,  6786,   676,

In [43]:
generation_config = GenerationConfig(max_new_tokens=100)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample = True, temperature = 0.1)
# generation_config = GenerationConfig(max_new_tokens=100, do_sample = True, temperature = 0.5)
# generation_config = GenerationConfig(max_new_tokens=100, do_sample = True, temperature = 1.0)

summary = dataset['test'][example_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
print("input:\n ", inputs, '\n\n\n')
# print("input_ids:\n ", inputs["input_ids"])
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config = generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash)
print('Example ', i+1)
print(dash)
print(f'INPUT PROMPT:\n{few_shot_prompt}')
# print(dataset['test'][index]['dialogue'])
print(dash)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash)
print(f'MODEL GENERATION - WITH ONE SHOT INFERENCE:\n{output}\n')


input:
  {'input_ids': tensor([[ 5267, 10384,    10,  1713,   345, 13515,   536,  4663,    10,   363,
            97,    19,    34,     6,  3059,    58,  1713,   345, 13515,   357,
          4663,    10,  1142,     3,     9,  1962,     5,    94,    31,     7,
             3,   324,    12,  4169,    57,    82,  1605,     5,  1713,   345,
         13515,   536,  4663,    10,    27,     7,    34,    58,    27,   141,
           150,   800,    34,    47,    78,  1480,     5,    27,   398,    36,
           326,   230,     5,  1713,   345, 13515,   357,  4663,    10,   363,
            31,     7,     8, 23601,    58,  1713,   345, 13515,   536,  4663,
            10,    27,   398,  3579,     8,  4169,    18,    17,  9288,    17,
            63,  2412,     5,  1713,   345, 13515,   357,  4663,    10,   148,
            31,   162,  2500,    13,    97,   780,     5,    37, 14421,  2478,
            19,   182,   885,     5,    94,   751,    31,    17,   240,    72,
           145,  6786,   676,